# CIFAR Conv Net

И так, в этом ноутбуке Вы сделаете превую в своей жизни сверточную сеть! На сложном датасете. Cкачайте его кстати, 

In [7]:
!mkdir cifar10
!curl -o cifar-10-python.tar.gz https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xvzf cifar-10-python.tar.gz -C cifar10

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  162M  100  162M    0     0  1905k      0  0:01:27  0:01:27 --:--:-- 4158kk      0  0:03:30  0:00:13  0:03:17  770k
cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [1]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
sys.path.append('/home/ec2-user/papers/preactivation_and_wide_resnet/')
import models

ImportError: No module named cuda

In [ ]:
from cifar import load_CIFAR10
plt.rcParams['figure.figsize'] = (10.0, 8.0) 

cifar10_dir = './cifar10/cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

In [ ]:
try:
    mean_std = np.load(os.path.join(os.getcwd(), 'cifar-10-mean_std.npz'))
    mean = mean_std['mean']
    std = mean_std['std']
except IOError:
    mean = (X_train.mean(axis=(0, 2, 3), keepdims=True).astype(np.float32) +\
        X_test.mean(axis=(0, 2, 3), keepdims=True).astype(np.float32)) / 2 
    std = np.concatenate((X_train, X_test)).std(axis=(0, 2, 3), keepdims=True).astype(np.float32)
    np.savez(os.path.join(os.getcwd(), 'cifar-10-mean_std.npz'),
             mean=mean, std=std)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8').transpose(1, 2, 0))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
import lasagne
from theano import tensor as T
from lasagne.nonlinearities import *

input_X = T.tensor4("X")
target_y = T.vector("target Y integer",dtype='int32')

    # Соберите нейронку: 
- Many times x (Conv+Pool)
- Many small convolutions like 3x3
- Baetch Norm 
- Residual Connection
- Data Augmentation 
- Learning rate Schedule 
- ...

### Для вдохновения 
- http://torch.ch/blog/2015/07/30/cifar.html
- http://www.robots.ox.ac.uk/~vgg/research/very_deep/
- https://www.cs.unc.edu/~wliu/papers/GoogLeNet.pdf
- https://github.com/szagoruyko/wide-residual-networks 

### Самое интересное
- Для сдачи задания нужно набрать на точность тесте > **92.5**% (это займет много времени, торопитесь :) )
- Для получения бонусных баллов > **95.0**%
- Будет очень хорошо если вы придумаете свою архитектуру или сможете обучить что-то из вышеперечисленного :)
- А для обучения всего этого добра вам будет куда удобнее использовать GPU на Amazon 
    - Инструкция https://github.com/persiyanov/ml-mipt/tree/master/amazon-howto 
    - Вам помогут tmux, CuDNN, ssh tunnel, nvidia-smi, ... 
    - Wish you get fun :)

In [4]:
# net = lasagne.layers.InputLayer(shape=(None, 3, 32, 32), input_var=input_X)
import models
network = models.ResNet_FullPre_Wide(input_var=input_X)
# import implemented2
# network= Deep_Residual_Learning_CIFAR.build_cnn(input_var=input_X)
# network = implemented2.create_v3(input_X, input_shape=(None, 3, 32, 32),ccp_num_filters=[64, 128, 128, 256],
#                                  fc_num_units=[256, 256])
# net = lasagne.layers.DenseLayer(net,num_units = 10, nonlinearity=softmax)

In [ ]:
output_train = lasagne.layers.get_output(network)
output_test = lasagne.layers.get_output(network, deterministic=True)

# set up the loss that we aim to minimize when using cat cross entropy our Y should be ints not one-hot
loss = lasagne.objectives.categorical_crossentropy(output_train, Y)
loss = loss.mean()

# if using ResNet use L2 regularization
all_layers = lasagne.layers.get_all_layers(output_layer)
l2_penalty = lasagne.regularization.regularize_layer_params(all_layers, lasagne.regularization.l2) * 0.0001
loss = loss + l2_penalty

# set up loss functions for validation dataset
test_loss = lasagne.objectives.categorical_crossentropy(output_test, Y)
test_loss = test_loss.mean()

test_acc = T.mean(T.eq(T.argmax(output_test, axis=1), Y), dtype=theano.config.floatX)

# get parameters from network and set up sgd with nesterov momentum to update parameters, l_r is shared var so it can be changed
l_r = theano.shared(np.array(LR_SCHEDULE[0], dtype=theano.config.floatX))
params = lasagne.layers.get_all_params(output_layer, trainable=True)
updates = nesterov_momentum(loss, params, learning_rate=l_r, momentum=0.9)
#updates = adam(loss, params, learning_rate=l_r)

# set up training and prediction functions
train_fn = theano.function(inputs=[X,Y], outputs=loss, updates=updates, allow_input_downcast=True)
valid_fn = theano.function(inputs=[X,Y], outputs=[test_loss, test_acc], allow_input_downcast=True)


### Вот и всё, пошли её учить

In [10]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [1]:
def augment_minibatches(minibatches):
    for inputs, targets in minibatches:
        yield augment_batch(inputs), targets

In [2]:
def augment_batch(X_batch, y_batch):  
    trans_1 = random.randint(0, (PAD_CROP*2))
    trans_2 = random.randint(0, (PAD_CROP*2))
    crop_x1 = trans_1
    crop_x2 = (PIXELS + trans_1)
    crop_y1 = trans_2
    crop_y2 = (PIXELS + trans_2)

    # flip left-right choice
    flip_lr = random.randint(0,1)

    # set empty copy to hold augmented images so that we don't overwrite
    X_batch_aug = np.copy(X_batch)

    # for each image in the batch do the augmentation
    for j in range(X_batch.shape[0]):
        # for each image channel
        for k in range(X_batch.shape[1]):
            # pad and crop images
            img_pad = np.pad(X_batch_aug[j,k], pad_width=((PAD_CROP,PAD_CROP), (PAD_CROP,PAD_CROP)), mode='constant')
            X_batch_aug[j,k] = img_pad[crop_x1:crop_x2, crop_y1:crop_y2]

            # flip left-right if chosen
            if flip_lr == 1:
                X_batch_aug[j,k] = np.fliplr(X_batch_aug[j,k])
    return X_batch_aug, y_batch

In [11]:
import augment2
aug1 = augment2.AddElementwise()
aug2 = augment2.Fliplr()
aug3  =augment2.Flipud()

In [21]:
def send(text):
    import requests
    requests.get("https://api.telegram.org/bot224136417:AAGJ2kKaoPiiksBVJ-8AslDTavFDL6btqqE/sendMessage?chat_id=153592892&text=%s" % text)

In [19]:
# learning_rate_var.set_value(lasagne.utils.floatX(learning_rate_var.get_value() * 0.01))

In [ ]:
from models import ResNet_FullPre_Wide as ResNet
#else:
    #print 'Unsupported model %s'%variant

# training params
LR_SCHEDULE = {
    0: 0.01,
    10: 0.1,
    80: 0.01,
    120: 0.001,
}

# Процесс обучения

In [ ]:
import time
X_val = X_test
y_val = y_test

num_epochs = 200 #количество проходов по данным

batch_size = 64 #размер мини-батча

train_eval = []
valid_eval = []
valid_acc = []
best_acc = 0.0

for epoch in range(num_epochs):
    if epoch in LR_SCHEDULE:
        l_r.set_value(LR_SCHEDULE[epoch])
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()

    for batch in augment_minibatches(iterate_minibatches(X_train, y_train,batch_size)):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1
        
#     for batch in aug3.augment_batches(iterate_minibatches(X_train, y_train,batch_size)):
#         inputs, targets = batch
#         train_err_batch, train_acc_batch= train_fun(inputs, targets)
#         train_err += train_err_batch
#         train_acc += train_acc_batch
#         train_batches += 1
#     for batch in aug1.augment_batches(iterate_minibatches(X_train, y_train,batch_size)):
#         inputs, targets = batch
#         train_err_batch, train_acc_batch= train_fun(inputs, targets)
#         train_err += train_err_batch
#         train_acc += train_acc_batch
#         train_batches += 1

    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1
    if val_acc > best_acc:
        best_acc = val_acc
        best_params = lasagne.layers.get_all_param_values(network)

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(val_acc / val_batches * 100))

In [35]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.4f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 80:
    print "Achievement unlocked: колдун 80 уровня"
else:
    print "Нужно больше магии!"

Final results:
  test accuracy:		0.9533 %
Achievement unlocked: колдун 80 уровня


## Заполните форму

https://goo.gl/forms/EeadABISlVmdJqgr2 